In [1]:
import pickle
import os
import re
import numpy as np
from gensim.models import word2vec
import logging
import pandas as pd

In [2]:
import sys
sys.path.insert(0, '../Core-scripts/')

from parse_and_prepare import ProteinProteinInteractionClassifier as ppi
import file_readers as fr
import prediction as pred

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

In [4]:
random_seeds = [144, 235, 905, 2895, 3462, 4225, 5056, 5192, 7751, 7813]

In [5]:
# Yeast using old model (w/o skipgramms and hierarchical softmax)

yeast_strict = pickle.load(open('../../Results/Yeast/yeast_mentions_strict_real.pkl', 'rb'))
yeast_gen = pickle.load(open('../../Results/Yeast/yeast_mentions_gen_real.pkl', 'rb'))
yeast_be = pickle.load(open('../../Results/Yeast/yeast_mentions_be_real.pkl', 'rb'))

In [18]:
yeast_strict_model = pred.make_w2v_model(yeast_strict, 'yeast_strict')
yeast_gen_model = pred.make_w2v_model(yeast_gen, 'yeast_gen')
yeast_be_model = pred.make_w2v_model(yeast_be, 'yeast_be')

2017-06-09 10:47:12,920 : INFO : collecting all words and their counts
2017-06-09 10:47:12,921 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-09 10:47:12,962 : INFO : collected 11431 word types from a corpus of 188505 raw words and 7145 sentences
2017-06-09 10:47:12,964 : INFO : Loading a fresh vocabulary
2017-06-09 10:47:12,973 : INFO : min_count=7 retains 2632 unique words (23% of original 11431, drops 8799)
2017-06-09 10:47:12,974 : INFO : min_count=7 leaves 171509 word corpus (90% of original 188505, drops 16996)
2017-06-09 10:47:12,983 : INFO : deleting the raw counts dictionary of 11431 items
2017-06-09 10:47:12,984 : INFO : sample=0.001 downsamples 42 most-common words
2017-06-09 10:47:12,985 : INFO : downsampling leaves estimated 121427 word corpus (70.8% of prior 171509)
2017-06-09 10:47:12,986 : INFO : estimated required memory for 2632 words and 800 dimensions: 18160800 bytes
2017-06-09 10:47:12,993 : INFO : resetting layer weights
2017-06

Parsing datasets sentences
Training Word2Vec Model


2017-06-09 10:47:14,060 : INFO : PROGRESS: at 87.95% examples, 530635 words/s, in_qsize 11, out_qsize 1
2017-06-09 10:47:14,092 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-06-09 10:47:14,095 : INFO : worker thread finished; awaiting finish of 6 more threads
2017-06-09 10:47:14,109 : INFO : worker thread finished; awaiting finish of 5 more threads
2017-06-09 10:47:14,115 : INFO : worker thread finished; awaiting finish of 4 more threads
2017-06-09 10:47:14,123 : INFO : worker thread finished; awaiting finish of 3 more threads
2017-06-09 10:47:14,137 : INFO : worker thread finished; awaiting finish of 2 more threads
2017-06-09 10:47:14,145 : INFO : worker thread finished; awaiting finish of 1 more threads
2017-06-09 10:47:14,146 : INFO : worker thread finished; awaiting finish of 0 more threads
2017-06-09 10:47:14,147 : INFO : training on 942525 raw words (606864 effective words) took 1.1s, 555259 effective words/s
2017-06-09 10:47:14,148 : INFO : precomputing

Parsing datasets sentences


2017-06-09 10:47:14,806 : INFO : collecting all words and their counts
2017-06-09 10:47:14,807 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-09 10:47:14,859 : INFO : PROGRESS: at sentence #10000, processed 242832 words, keeping 11271 word types
2017-06-09 10:47:14,911 : INFO : PROGRESS: at sentence #20000, processed 485407 words, keeping 15040 word types
2017-06-09 10:47:14,972 : INFO : PROGRESS: at sentence #30000, processed 726685 words, keeping 17636 word types


...

2017-06-09 10:47:15,028 : INFO : PROGRESS: at sentence #40000, processed 970059 words, keeping 19711 word types
2017-06-09 10:47:15,076 : INFO : collected 21147 word types from a corpus of 1155938 raw words and 47662 sentences
2017-06-09 10:47:15,076 : INFO : Loading a fresh vocabulary
2017-06-09 10:47:15,102 : INFO : min_count=7 retains 6774 unique words (32% of original 21147, drops 14373)
2017-06-09 10:47:15,103 : INFO : min_count=7 leaves 1127019 word corpus (97% of original 1155938, drops 28919)
2017-06-09 10:47:15,121 : INFO : deleting the raw counts dictionary of 21147 items
2017-06-09 10:47:15,123 : INFO : sample=0.001 downsamples 48 most-common words
2017-06-09 10:47:15,123 : INFO : downsampling leaves estimated 821525 word corpus (72.9% of prior 1127019)
2017-06-09 10:47:15,124 : INFO : estimated required memory for 6774 words and 800 dimensions: 46740600 bytes
2017-06-09 10:47:15,151 : INFO : resetting layer weights
2017-06-09 10:47:15,284 : INFO : training model with 8 work

Parsing datasets sentences


2017-06-09 10:47:29,261 : INFO : collecting all words and their counts
2017-06-09 10:47:29,262 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-06-09 10:47:29,303 : INFO : PROGRESS: at sentence #10000, processed 229190 words, keeping 12513 word types
2017-06-09 10:47:29,344 : INFO : PROGRESS: at sentence #20000, processed 459442 words, keeping 16953 word types
2017-06-09 10:47:29,386 : INFO : PROGRESS: at sentence #30000, processed 689224 words, keeping 20231 word types
2017-06-09 10:47:29,428 : INFO : PROGRESS: at sentence #40000, processed 919147 words, keeping 22898 word types


Training Word2Vec Model


2017-06-09 10:47:29,469 : INFO : PROGRESS: at sentence #50000, processed 1147837 words, keeping 25161 word types
2017-06-09 10:47:29,513 : INFO : PROGRESS: at sentence #60000, processed 1376264 words, keeping 27203 word types
2017-06-09 10:47:29,555 : INFO : PROGRESS: at sentence #70000, processed 1604727 words, keeping 29052 word types
2017-06-09 10:47:29,597 : INFO : PROGRESS: at sentence #80000, processed 1833441 words, keeping 30749 word types
2017-06-09 10:47:29,639 : INFO : PROGRESS: at sentence #90000, processed 2063303 words, keeping 32266 word types
2017-06-09 10:47:29,681 : INFO : PROGRESS: at sentence #100000, processed 2292153 words, keeping 33648 word types
2017-06-09 10:47:29,724 : INFO : PROGRESS: at sentence #110000, processed 2520925 words, keeping 34979 word types
2017-06-09 10:47:29,766 : INFO : PROGRESS: at sentence #120000, processed 2748644 words, keeping 36179 word types
2017-06-09 10:47:29,808 : INFO : PROGRESS: at sentence #130000, processed 2976311 words, keep

In [7]:
from sklearn.ensemble import RandomForestClassifier
def RF_classifier_bow(train_vector, test_vector, labels_train, labels_test, name, n_trees=100, ran_s=23):
    """Adding Later"""
    print ("Training the random forest...")

    # Initialize a Random Forest classifier with n_trees trees
    forest = RandomForestClassifier(n_estimators=int(n_trees), random_state=int(ran_s))

    # Fit the forest to the training set, using the bag of words as
    # features and the sentiment labels as the response variable
    #
    # This may take a few minutes to run
    forest = forest.fit(train_vector, labels_train)
    probs = forest.predict_proba(test_vector)[:, 1] #For ROC Curve

    # Use the random forest to make sentiment label predictions
    print ("Predicting based on model...")
    result = forest.predict(test_vector)
    
    #Metrics
    accuracy = metrics.accuracy_score(labels_test, result)
    roc_auc = metrics.roc_auc_score(labels_test, probs)
    class_report = metrics.classification_report(labels_test, result)
    
    #Print Model report:
    print(name, '\nModel Report')
    print(name, 'Accuracy: %.4g' % accuracy)
    print(name, 'AUC Score (Train): %f' % roc_auc)
    print(name, 'Report \n', class_report)
    
    return accuracy, roc_auc, result

In [8]:
def XGB_modelfit(alg, train_vecs, train_labels, test_vecs, test_labels, w2v_model_type, useTrainCV=True, cv_folds=5, feature_selection=False):

    if feature_selection:
        clf = ExtraTreesClassifier(n_estimators=400)
        clf = clf.fit(train_vecs, train_labels)
        model = SelectFromModel(clf, prefit=True)
        train_vecs = model.transform(train_vecs)
        test_vecs = model.transform(test_vecs)

#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(train_vecs,
#                               label=train_labels)
#         cvresult = xgb.cv(xgb_param,
#                           xgtrain,
#                           num_boost_round=alg.get_params()['n_estimators'],
#                           nfold=cv_folds,
#                           metrics='auc',
#                           early_stopping_rounds=50)
#         alg.set_params(n_estimators=cvresult.shape[0])

    #fit the algorithm on the data
    alg.fit(train_vecs, train_labels, eval_metric='auc')

    #Predict training set:
    test_preds = alg.predict(test_vecs)
    test_predictions = [round(val) for val in test_preds]
    test_predprob = alg.predict_proba(test_vecs)[:, 1]

    #Metrics
    accuracy = metrics.accuracy_score(test_labels, test_predictions)
    roc_auc = metrics.roc_auc_score(test_labels, test_predprob)
    class_report = metrics.classification_report(test_labels, test_predictions)

    #Print Model report:
    print(w2v_model_type, '\nModel Report')
    print(w2v_model_type, 'Accuracy: %.4g' % accuracy)
    print(w2v_model_type, 'AUC Score (Train): %f' % roc_auc)
    print(w2v_model_type, 'Report \n', class_report)
    
    return accuracy, roc_auc, test_predictions

In [9]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

rf_accuracy = []
rf_auc_score = []
rf_pred_labels = []
xgb_accuracy = []
xgb_auc_score = []
xgb_pred_labels = []
true_test_labels = []
    
for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)

    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        xgb_accu, xgb_auc, xgb_labels = XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        rf_accu, rf_auc, rf_labels = RF_classifier_bow(entry[0], entry[1], entry[4], entry[5], 'BOW RF '+str(seed))
        xgb_accuracy.append(xgb_accu)
        xgb_auc_score.append(xgb_auc)
        xgb_pred_labels.extend(xgb_labels)
        rf_accuracy.append(rf_accu)
        rf_auc_score.append(rf_auc)
        rf_pred_labels.extend(rf_labels)
        true_test_labels += entry[5]
        
rf_avg_recall = metrics.recall_score(true_test_labels, rf_pred_labels, average='micro')
rf_avg_precision = metrics.precision_score(true_test_labels, rf_pred_labels, average='micro')
xgb_avg_recall = metrics.recall_score(true_test_labels, xgb_pred_labels, average='micro')
xgb_avg_precision = metrics.precision_score(true_test_labels, xgb_pred_labels, average='micro')

print('Mean RF accuracy - ', np.mean(rf_accuracy))
print('Mean RF AUC - ', np.mean(rf_auc_score))
print('Mean RF Recall - ', rf_avg_recall)
print('Mean RF Precision - ', rf_avg_precision)
print('Mean XGB accuracy - ', np.mean(xgb_accuracy))
print('Mean XGB AUC - ', np.mean(xgb_auc_score))
print('Mean XGB Recall - ', xgb_avg_recall)
print('Mean XGB Precision - ', xgb_avg_precision)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.6111
BOW XGB 144 AUC Score (Train): 0.651786
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.78      0.69      0.73        42
          1       0.24      0.33      0.28        12

avg / total       0.66      0.61      0.63        54

Training the random forest...
Predicting based on model...
BOW RF 144 
Model Report
BOW RF 144 Accuracy: 0.5
BOW RF 144 AUC Score (Train): 0.519841
BOW RF 144 Report 
              precision    recall  f1-score   support

          0       0.78      0.50      0.61        42
          1       0.22      0.50      0.31        12

avg / total       0.65      0.50      0.54        54

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.6707
BOW XGB 235 AUC Score (Train): 0.722120
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.57      0.82      0.67        34
          1       0.82      0.56      0.67        48

avg / total       0.72      0.67      0.67        82

Training the random forest...
Predicting based on model...
BOW RF 235 
Model Report
BOW RF 235 Accuracy: 0.7073
BOW RF 235 AUC Score (Train): 0.742647
BOW RF 235 Report 
              precision    recall  f1-score   support

          0       0.62      0.74      0.68        34
          1       0.79      0.69      0.73        48

avg / total       0.72      0.71      0.71        82

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 905 
Model Report
BOW XGB 905 Accuracy: 0.75
BOW XGB 905 AUC Score (Train): 0.629032
BOW XGB 905 Report 
              precision    recall  f1-score   support

          0       0.73      0.94      0.82        49
          1       0.82      0.45      0.58        31

avg / total       0.77      0.75      0.73        80

Training the random forest...
Predicting based on model...
BOW RF 905 
Model Report
BOW RF 905 Accuracy: 0.5875
BOW RF 905 AUC Score (Train): 0.585912
BOW RF 905 Report 
              precision    recall  f1-score   support

          0       0.66      0.67      0.67        49
          1       0.47      0.45      0.46        31

avg / total       0.59      0.59      0.59        80

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.6364
BOW XGB 5192 AUC Score (Train): 0.675926
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.62      0.89      0.73        36
          1       0.71      0.33      0.45        30

avg / total       0.66      0.64      0.60        66

Training the random forest...
Predicting based on model...
BOW RF 5192 
Model Report
BOW RF 5192 Accuracy: 0.6515
BOW RF 5192 AUC Score (Train): 0.713889
BOW RF 5192 Report 
              precision    recall  f1-score   support

          0       0.66      0.75      0.70        36
          1       0.64      0.53      0.58        30

avg / total       0.65      0.65      0.65        66

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.5818
BOW XGB 7751 AUC Score (Train): 0.675926
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.55      0.85      0.67        27
          1       0.69      0.32      0.44        28

avg / total       0.62      0.58      0.55        55

Training the random forest...
Predicting based on model...
BOW RF 7751 
Model Report
BOW RF 7751 Accuracy: 0.6182
BOW RF 7751 AUC Score (Train): 0.711640
BOW RF 7751 Report 
              precision    recall  f1-score   support

          0       0.62      0.59      0.60        27
          1       0.62      0.64      0.63        28

avg / total       0.62      0.62      0.62        55

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.8101
BOW XGB 7813 AUC Score (Train): 0.724176
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.86      0.92      0.89        65
          1       0.44      0.29      0.35        14

avg / total       0.78      0.81      0.79        79

Training the random forest...
Predicting based on model...
BOW RF 7813 
Model Report
BOW RF 7813 Accuracy: 0.7595
BOW RF 7813 AUC Score (Train): 0.722527
BOW RF 7813 Report 
              precision    recall  f1-score   support

          0       0.90      0.80      0.85        65
          1       0.38      0.57      0.46        14

avg / total       0.81      0.76      0.78        79

Mean RF accuracy -  0.639207836895
Mean RF AUC -  0.677435381841
Mean RF Recall -  0.641959798995
Mean RF Precision -  0.641959798995
Mean XGB accuracy -  0.666034575412
Mean XGB AUC -  0.688446630788
Mean XGB Recall -  0.66959798995
Mean XGB Precision -  0.6695

In [19]:
import time
import numpy as np
name_of_result = 'yeast_W2V'
strict_data = yeast_strict
w2v_strict = yeast_strict_model
xgb_clf = XGBClassifier(seed=24)
start = time.time()

bow_accuracy = []
bow_auc_score = []
bow_pred_labels = []
w2v_accuracy = []
w2v_auc_score = []
w2v_pred_labels = []
true_test_labels = []

for seed in random_seeds:
    strict_list_SR = pred.make_models(strict_data,
                                      name_of_result+'_SR_'+str(seed),
                                      prev_model=w2v_strict,
                                      ran_state=seed,
                                      BOW=True)

    strict_final_list = [strict_list_SR]
    
    print ('\nPredicting\n')

    for entry in strict_final_list:
        bow_accu, bow_auc, bow_labels, _, _ = pred.XGB_modelfit(xgb_clf, entry[0], entry[4], entry[1], entry[5], 'BOW XGB '+str(seed))
        w2v_accu, w2v_auc, w2v_labels, _, _  = pred.XGB_modelfit(xgb_clf, entry[2], entry[4], entry[3], entry[5], 'W2V XGB '+str(seed))
        w2v_accuracy.append(w2v_accu)
        w2v_auc_score.append(w2v_auc)
        w2v_pred_labels.extend(w2v_labels)
        bow_accuracy.append(bow_accu)
        bow_auc_score.append(bow_auc)
        bow_pred_labels.extend(bow_labels)
        true_test_labels += entry[5]
        
bow_avg_recall = metrics.recall_score(true_test_labels, bow_pred_labels, average='micro')
bow_avg_precision = metrics.precision_score(true_test_labels, bow_pred_labels, average='micro')
w2v_avg_recall = metrics.recall_score(true_test_labels, w2v_pred_labels, average='micro')
w2v_avg_precision = metrics.precision_score(true_test_labels, w2v_pred_labels, average='micro')

print('Mean BOW accuracy - ', np.mean(bow_accuracy))
print('Mean BOW AUC - ', np.mean(bow_auc_score))
print('Mean BOW Recall - ', bow_avg_recall)
print('Mean BOW Precision - ', bow_avg_precision)
print('Mean W2V accuracy - ', np.mean(w2v_accuracy))
print('Mean W2V AUC - ', np.mean(w2v_auc_score))
print('Mean W2V Recall - ', w2v_avg_recall)
print('Mean W2V Precision - ', w2v_avg_precision)

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles


Predicting

BOW XGB 144 
Model Report
BOW XGB 144 Accuracy: 0.6111
BOW XGB 144 AUC Score (Train): 0.651786
BOW XGB 144 Report 
              precision    recall  f1-score   support

          0       0.78      0.69      0.73        42
          1       0.24      0.33      0.28        12

avg / total       0.66      0.61      0.63        54

W2V XGB 144 
Model Report
W2V XGB 144 Accuracy: 0.6111
W2V XGB 144 AUC Score (Train): 0.615079
W2V XGB 144 Report 
              precision    recall  f1-score   support

          0       0.89      0.57      0.70        42
          1       0.33      0.75      0.46        12

avg / total       0.77      0.61      0.64        54

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 235 
Model Report
BOW XGB 235 Accuracy: 0.622
BOW XGB 235 AUC Score (Train): 0.731005
BOW XGB 235 Report 
              precision    recall  f1-score   support

          0       0.53      0.85      0.65        34
          1       0.81      0.46      0.59        48

avg / total       0.70      0.62      0.61        82

W2V XGB 235 
Model Report
W2V XGB 235 Accuracy: 0.5732
W2V XGB 235 AUC Score (Train): 0.683824
W2V XGB 235 Report 
              precision    recall  f1-score   support

          0       0.49      0.74      0.59        34
          1       0.71      0.46      0.56        48

avg / total       0.62      0.57      0.57        82

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 905 
Model Report
BOW XGB 905 Accuracy: 0.725
BOW XGB 905 AUC Score (Train): 0.681369
BOW XGB 905 Report 
              precision    recall  f1-score   support

          0       0.70      0.98      0.81        49
          1       0.91      0.32      0.48        31

avg / total       0.78      0.72      0.68        80

W2V XGB 905 
Model Report
W2V XGB 905 Accuracy: 0.65
W2V XGB 905 AUC Score (Train): 0.622778
W2V XGB 905 Report 
              precision    recall  f1-score   support

          0       0.70      0.76      0.73        49
          1       0.56      0.48      0.52        31

avg / total       0.64      0.65      0.64        80

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



...


Predicting

BOW XGB 5192 
Model Report
BOW XGB 5192 Accuracy: 0.6061
BOW XGB 5192 AUC Score (Train): 0.639352
BOW XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.59      0.92      0.72        36
          1       0.70      0.23      0.35        30

avg / total       0.64      0.61      0.55        66

W2V XGB 5192 
Model Report
W2V XGB 5192 Accuracy: 0.6212
W2V XGB 5192 AUC Score (Train): 0.648148
W2V XGB 5192 Report 
              precision    recall  f1-score   support

          0       0.63      0.75      0.68        36
          1       0.61      0.47      0.53        30

avg / total       0.62      0.62      0.61        66

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7751 
Model Report
BOW XGB 7751 Accuracy: 0.5636
BOW XGB 7751 AUC Score (Train): 0.666005
BOW XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.54      0.81      0.65        27
          1       0.64      0.32      0.43        28

avg / total       0.59      0.56      0.54        55

W2V XGB 7751 
Model Report
W2V XGB 7751 Accuracy: 0.6909
W2V XGB 7751 AUC Score (Train): 0.768519
W2V XGB 7751 Report 
              precision    recall  f1-score   support

          0       0.67      0.74      0.70        27
          1       0.72      0.64      0.68        28

avg / total       0.69      0.69      0.69        55

Cleaning and parsing the training set articles

Cleaning and parsing the testing set articles



../Core-scripts/prediction.py:187: RuntimeWarning: invalid value encountered in true_divide
  feature_vec = np.divide(feature_vec,nwords)



Predicting

BOW XGB 7813 
Model Report
BOW XGB 7813 Accuracy: 0.8354
BOW XGB 7813 AUC Score (Train): 0.715934
BOW XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.86      0.95      0.91        65
          1       0.57      0.29      0.38        14

avg / total       0.81      0.84      0.81        79

W2V XGB 7813 
Model Report
W2V XGB 7813 Accuracy: 0.8228
W2V XGB 7813 AUC Score (Train): 0.820879
W2V XGB 7813 Report 
              precision    recall  f1-score   support

          0       0.88      0.91      0.89        65
          1       0.50      0.43      0.46        14

avg / total       0.81      0.82      0.82        79

Mean BOW accuracy -  0.651130700936
Mean BOW AUC -  0.686714831161
Mean BOW Recall -  0.654522613065
Mean BOW Precision -  0.654522613065
Mean W2V accuracy -  0.656886438077
Mean W2V AUC -  0.688678739329
Mean W2V Recall -  0.654522613065
Mean W2V Precision -  0.654522613065


In [21]:
a = [rf_accuracy, rf_auc_score, rf_pred_labels, 
     bow_accuracy, bow_auc_score, bow_pred_labels, 
     w2v_accuracy, w2v_auc_score, w2v_pred_labels,
     true_test_labels]
pickle.dump(a, open('BOW_vs_W2v.pkl', 'wb'))

[0.61111111111111116,
 0.67073170731707321,
 0.72499999999999998,
 0.62068965517241381,
 0.6166666666666667,
 0.59090909090909094,
 0.7021276595744681,
 0.60606060606060608,
 0.59999999999999998,
 0.83544303797468356]

In [14]:
c

[1, 2, 3, 2, 3, 4]